In [1]:
import pyAgrum as gum
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import itertools

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
pd.set_option('display.max_rows', 200)

In [2]:
files = os.listdir("outputs/scores")#[::-1]
# np.random.shuffle(files)
len(files)

12979

In [3]:
# %%time
# df = pd.DataFrame({"items":[],"nodes":[], "arcs_ratio":[], "timesteps":[], "values":[], "algorithm":[], "scorefunction":[], "fscore":[]})
# # for fi, file in enumerate(np.random.choice(files,10)):
# for fi, file in enumerate(files):
#     structure, rep = file[9:-4].split("_")
# #     bn = gum.BayesNet()
# #     bn.loadBIFXML(f"outputs/structures/structure{structure}_{rep}.bifxml")
    
#     for _,row in pd.read_csv(f"outputs/scores/structure{structure}_{rep}.csv").iterrows():
    
# #         data,_ = gum.generateSample(bn,row["items"],None,False)
# #         data = data.reindex(sorted(data.columns, key=lambda x: (len(x), int(x.split(".")[1]), x.split(".")[0])), axis=1)
# #         data = data.astype('int')

# #         cov = data.cov()

#         df = pd.concat([df,pd.DataFrame({"items":[row["items"]], "nodes":[row["nodes"]], "arcs_ratio":[row["arcs_ratio"]], "timesteps":[row["timesteps"]], "values":[row["values"]], "algorithm":[row["algorithm"]], "scorefunction":[row["score"]], "fscore":[row["fscore"]]})], ignore_index=True)
# df
df = pd.read_csv("outputs/scores2.csv")
df

def normalize(col):
    # TODO: GROTER DAN WAT????
    if max(col) > 1:
        return col/max(col)
    return col

columns = ['items', 'nodes', 'timesteps', 'arcs_ratio', 'islands', 'values', 'nodestates', 'arcs', 'items/nodestates', 'items/arcs']
df[columns] = df[columns].apply(normalize)
df

,Unnamed: 0,items,nodes,timesteps,arcs_ratio,islands,values,algorithm,scorefunction,recall,...,fscore,dist2opt,tp,tn,fp,fn,nodestates,arcs,items/nodestates,items/arcs
0,0,0.1,0.5,0.333333,0.499999,1,0.5,3OFF2,AIC,0.666667,...,0.800000,0.333333,2,9,0,1,0.000244,0.1875,0.100000,0.1000
1,1,0.1,0.5,0.333333,0.499999,1,0.5,3OFF2,AIC,0.000000,...,0.000000,1.414214,0,7,2,3,0.000244,0.1875,0.100000,0.1000
2,2,0.1,0.5,0.333333,0.499999,1,0.5,3OFF2,AIC,0.000000,...,0.000000,1.414214,0,7,2,3,0.000244,0.1875,0.100000,0.1000
3,3,0.1,0.5,0.333333,0.499999,1,0.5,3OFF2,AIC,0.333333,...,0.400000,0.833333,1,8,1,2,0.000244,0.1875,0.100000,0.1000
4,4,0.1,0.5,0.333333,0.499999,1,0.5,3OFF2,AIC,0.000000,...,0.000000,1.414214,0,7,2,3,0.000244,0.1875,0.100000,0.1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129785,129785,1.0,1.0,1.000000,1.000000,1,1.0,TABU,K2,0.527273,...,0.495726,0.711878,29,464,33,26,1.000000,1.0000,0.000244,0.1875
129786,129786,1.0,1.0,1.000000,1.000000,1,1.0,TABU,K2,0.672727,...,0.587302,0.580023,37,463,34,18,1.000000,1.0000,0.000244,0.1875
129787,129787,1.0,1.0,1.000000,1.000000,1,1.0,TABU,K2,0.890909,...,0.830508,0.247555,49,483,14,6,1.000000,1.0000,0.000244,0.1875
129788,129788,1.0,1.0,1.000000,1.000000,1,1.0,TABU,K2,1.000000,...,0.880000,0.214286,55,482,15,0,1.000000,1.0000,0.000244,0.1875


# One hot Encoding

In [4]:
df1 = pd.get_dummies(df, columns=["algorithm","scorefunction"], dtype=int)
df1

,Unnamed: 0,items,nodes,timesteps,arcs_ratio,islands,values,recall,precision,fscore,...,algorithm_3OFF2,algorithm_HC,algorithm_MIIC,algorithm_MMHC,algorithm_TABU,scorefunction_AIC,scorefunction_BDEU,scorefunction_BIC,scorefunction_K2,scorefunction_L2L
0,0,0.1,0.5,0.333333,0.499999,1,0.5,0.666667,1.000000,0.800000,...,1,0,0,0,0,1,0,0,0,0
1,1,0.1,0.5,0.333333,0.499999,1,0.5,0.000000,0.000000,0.000000,...,1,0,0,0,0,1,0,0,0,0
2,2,0.1,0.5,0.333333,0.499999,1,0.5,0.000000,0.000000,0.000000,...,1,0,0,0,0,1,0,0,0,0
3,3,0.1,0.5,0.333333,0.499999,1,0.5,0.333333,0.500000,0.400000,...,1,0,0,0,0,1,0,0,0,0
4,4,0.1,0.5,0.333333,0.499999,1,0.5,0.000000,0.000000,0.000000,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129785,129785,1.0,1.0,1.000000,1.000000,1,1.0,0.527273,0.467742,0.495726,...,0,0,0,0,1,0,0,0,1,0
129786,129786,1.0,1.0,1.000000,1.000000,1,1.0,0.672727,0.521127,0.587302,...,0,0,0,0,1,0,0,0,1,0
129787,129787,1.0,1.0,1.000000,1.000000,1,1.0,0.890909,0.777778,0.830508,...,0,0,0,0,1,0,0,0,1,0
129788,129788,1.0,1.0,1.000000,1.000000,1,1.0,1.000000,0.785714,0.880000,...,0,0,0,0,1,0,0,0,1,0


In [5]:
%%time
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    combinations = list(itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(1,len(s)+1)))
    
    merged = []
    for combination in combinations:
        merged.append(np.concatenate(combination, axis=None))
    return merged
    
item_c = ["items"]
nodes_c = ["nodes"]
arc_c = ["arcs_ratio"]
times_c = ["timesteps"]
values_c = ["values"]
algorithm_c = ["algorithm_3OFF2","algorithm_HC","algorithm_MIIC","algorithm_MMHC","algorithm_TABU"]
score_c = ["scorefunction_AIC","scorefunction_BDEU","scorefunction_BIC","scorefunction_K2","scorefunction_L2L"]
nodestates = ["nodestates"]
arcs = ["arcs"]
items_nodestates = ["items/nodestates"]
items_arcs = ["items/arcs"]


columns_c = [item_c, nodes_c, arc_c, times_c, values_c, algorithm_c, score_c, nodestates, arcs, items_nodestates, items_arcs]

column_sets = powerset(columns_c)

results = pd.DataFrame()
for columns in column_sets:
    # Splitting the data into training and testing data
    X_train, X_test, y_train, y_test = train_test_split(df1[columns], df1["fscore"], test_size = 0.2)

    regr = RandomForestRegressor()

    regr.fit(X_train, y_train)
        
    results =  pd.concat([results, pd.DataFrame({"items":["items" in columns], "nodes":["nodes" in columns], "arcs_ratio":["arcs_ratio" in columns], "timesteps":["timesteps" in columns], "values":["values" in columns], "algorithm":["algorithm_HC" in columns], "scorefunction":["scorefunction_AIC" in columns], "nodestates":["nodestates" in columns], "arcs":["arcs" in columns], "items/nodestates":["items/nodestates" in columns], "items/arcs":["items/arcs" in columns], "score":[regr.score(X_test, y_test)]})], ignore_index=True)

results

CPU times: total: 4h 48min 27s
Wall time: 4h 58min 49s


,items,nodes,arcs_ratio,timesteps,values,algorithm,scorefunction,nodestates,arcs,items/nodestates,items/arcs,score
0,True,False,False,False,False,False,False,False,False,False,False,0.006947
1,False,True,False,False,False,False,False,False,False,False,False,0.083690
2,False,False,True,False,False,False,False,False,False,False,False,0.007086
3,False,False,False,True,False,False,False,False,False,False,False,0.067500
4,False,False,False,False,True,False,False,False,False,False,False,-0.000146
...,...,...,...,...,...,...,...,...,...,...,...,...
2042,True,True,True,False,True,True,True,True,True,True,True,0.328131
2043,True,True,False,True,True,True,True,True,True,True,True,0.389420
2044,True,False,True,True,True,True,True,True,True,True,True,0.387605
2045,False,True,True,True,True,True,True,True,True,True,True,0.394443


In [6]:
cm = sns.light_palette("red", as_cmap=True)

results["items"] = results["items"].astype(int)
results["nodes"] = results["nodes"].astype(int)
results["arcs_ratio"] = results["arcs_ratio"].astype(int)
results["timesteps"] = results["timesteps"].astype(int)
results["values"] = results["values"].astype(int)
results["algorithm"] = results["algorithm"].astype(int)
results["scorefunction"] = results["scorefunction"].astype(int)
results["nodestates"] = results["nodestates"].astype(int)
results["arcs"] = results["arcs"].astype(int)
results["items/nodestates"] = results["items/nodestates"].astype(int)
results["items/arcs"] = results["items/arcs"].astype(int)

results.sort_values("score", ascending=False).to_csv("hotlinreg.csv")
results.sort_values("score", ascending=False).head(10).style.background_gradient(cmap='RdYlGn', vmin=0, vmax=1)

,items,nodes,arcs_ratio,timesteps,values,algorithm,scorefunction,nodestates,arcs,items/nodestates,items/arcs,score
1243,1,0,0,1,0,1,1,0,1,0,1,0.421401
1616,1,0,1,1,1,1,0,1,0,1,0,0.418523
717,1,0,0,1,0,1,1,0,1,0,0,0.418257
1167,1,0,1,1,0,1,1,0,0,0,1,0.417292
1097,1,1,0,1,0,1,1,0,0,0,1,0.416695
719,1,0,0,1,0,1,1,0,0,0,1,0.415277
1165,1,0,1,1,0,1,1,0,1,0,0,0.415038
1501,1,1,1,1,0,1,1,0,1,0,0,0.414881
1635,1,0,1,1,0,1,1,0,1,0,1,0.414798
1369,0,1,0,1,0,1,1,0,1,0,1,0.414765


# Discretized

In [7]:
df2 = df.copy()
df2['algorithmi'] = df2['algorithm'].map({'HC': 0, 'TABU':1, '3OFF2':2, 'MIIC':3, 'MMHC':4})
df2['scorefunctioni'] = df2['scorefunction'].map({'BDEU': 0, 'AIC':1, 'BIC':2, 'K2':3, 'L2L':4})
df2

,Unnamed: 0,items,nodes,timesteps,arcs_ratio,islands,values,algorithm,scorefunction,recall,...,tp,tn,fp,fn,nodestates,arcs,items/nodestates,items/arcs,algorithmi,scorefunctioni
0,0,0.1,0.5,0.333333,0.499999,1,0.5,3OFF2,AIC,0.666667,...,2,9,0,1,0.000244,0.1875,0.100000,0.1000,2,1
1,1,0.1,0.5,0.333333,0.499999,1,0.5,3OFF2,AIC,0.000000,...,0,7,2,3,0.000244,0.1875,0.100000,0.1000,2,1
2,2,0.1,0.5,0.333333,0.499999,1,0.5,3OFF2,AIC,0.000000,...,0,7,2,3,0.000244,0.1875,0.100000,0.1000,2,1
3,3,0.1,0.5,0.333333,0.499999,1,0.5,3OFF2,AIC,0.333333,...,1,8,1,2,0.000244,0.1875,0.100000,0.1000,2,1
4,4,0.1,0.5,0.333333,0.499999,1,0.5,3OFF2,AIC,0.000000,...,0,7,2,3,0.000244,0.1875,0.100000,0.1000,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129785,129785,1.0,1.0,1.000000,1.000000,1,1.0,TABU,K2,0.527273,...,29,464,33,26,1.000000,1.0000,0.000244,0.1875,1,3
129786,129786,1.0,1.0,1.000000,1.000000,1,1.0,TABU,K2,0.672727,...,37,463,34,18,1.000000,1.0000,0.000244,0.1875,1,3
129787,129787,1.0,1.0,1.000000,1.000000,1,1.0,TABU,K2,0.890909,...,49,483,14,6,1.000000,1.0000,0.000244,0.1875,1,3
129788,129788,1.0,1.0,1.000000,1.000000,1,1.0,TABU,K2,1.000000,...,55,482,15,0,1.000000,1.0000,0.000244,0.1875,1,3


In [8]:
%%time
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    combinations = list(itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(1,len(s)+1)))
    
    merged = []
    for combination in combinations:
        merged.append(np.concatenate(combination, axis=None))
    return merged
    
item_c = ["items"]
nodes_c = ["nodes"]
arc_c = ["arcs_ratio"]
times_c = ["timesteps"]
values_c = ["values"]
algorithm_c = ["algorithmi"]
score_c = ["scorefunctioni"]
nodestates = ["nodestates"]
arcs = ["arcs"]
items_nodestates = ["items/nodestates"]
items_arcs = ["items/arcs"]


columns_c = [item_c, nodes_c, arc_c, times_c, values_c, algorithm_c, score_c, nodestates, arcs, items_nodestates, items_arcs]


column_sets = powerset(columns_c)

results1 = pd.DataFrame()
for columns in column_sets:
    # Splitting the data into training and testing data
    X_train, X_test, y_train, y_test = train_test_split(df2[columns], df2["fscore"], test_size = 0.2)

    regr = RandomForestRegressor()

    regr.fit(X_train, y_train)
        
    results1 =  pd.concat([results1, pd.DataFrame({"items":["items" in columns], "nodes":["nodes" in columns], "arcs_ratio":["arcs_ratio" in columns], "timesteps":["timesteps" in columns], "values":["values" in columns], "algorithm":["algorithm_HC" in columns], "scorefunction":["scorefunction_AIC" in columns], "nodestates":["nodestates" in columns], "arcs":["arcs" in columns], "items/nodestates":["items/nodestates" in columns], "items/arcs":["items/arcs" in columns], "score":[regr.score(X_test, y_test)]})], ignore_index=True)

results1

CPU times: total: 2h 20min 10s
Wall time: 2h 20min 50s


,items,nodes,arcs_ratio,timesteps,values,algorithm,scorefunction,nodestates,arcs,items/nodestates,items/arcs,score
0,True,False,False,False,False,False,False,False,False,False,False,0.008625
1,False,True,False,False,False,False,False,False,False,False,False,0.085177
2,False,False,True,False,False,False,False,False,False,False,False,0.006739
3,False,False,False,True,False,False,False,False,False,False,False,0.065283
4,False,False,False,False,True,False,False,False,False,False,False,-0.000227
...,...,...,...,...,...,...,...,...,...,...,...,...
2042,True,True,True,False,True,False,False,True,True,True,True,0.320580
2043,True,True,False,True,True,False,False,True,True,True,True,0.380247
2044,True,False,True,True,True,False,False,True,True,True,True,0.396568
2045,False,True,True,True,True,False,False,True,True,True,True,0.386744


In [9]:
cm = sns.light_palette("red", as_cmap=True)

results1["items"] = results1["items"].astype(int)
results1["nodes"] = results1["nodes"].astype(int)
results1["arcs_ratio"] = results1["arcs_ratio"].astype(int)
results1["timesteps"] = results1["timesteps"].astype(int)
results1["values"] = results1["values"].astype(int)
results1["algorithm"] = results1["algorithm"].astype(int)
results1["scorefunction"] = results1["scorefunction"].astype(int)
results1["nodestates"] = results1["nodestates"].astype(int)
results1["arcs"] = results1["arcs"].astype(int)
results1["items/nodestates"] = results1["items/nodestates"].astype(int)
results1["items/arcs"] = results1["items/arcs"].astype(int)

results1.sort_values("score", ascending=False).to_csv("disclinreg.csv")
results1.sort_values("score", ascending=False).head(10).style.background_gradient(cmap='RdYlGn', vmin=0, vmax=1)

,items,nodes,arcs_ratio,timesteps,values,algorithm,scorefunction,nodestates,arcs,items/nodestates,items/arcs,score
1579,1,1,0,1,0,0,0,0,1,0,1,0.428716
1167,1,0,1,1,0,0,0,0,0,0,1,0.421570
1635,1,0,1,1,0,0,0,0,1,0,1,0.418919
1719,0,1,1,1,0,0,0,0,1,0,1,0.416379
724,1,0,0,1,0,0,0,0,1,0,1,0.415592
719,1,0,0,1,0,0,0,0,0,0,1,0.415486
991,0,0,0,1,0,0,0,0,1,0,1,0.415181
655,1,0,1,1,0,0,0,0,0,0,1,0.415089
845,0,1,0,1,0,0,0,0,0,0,1,0.414973
514,0,0,0,1,0,0,0,0,1,0,1,0.413536


# Bins - One Hot Encoding

In [10]:
df["fscore"] = pd.cut(df["fscore"],bins=[-0.01,0.2,0.4,0.6,0.8,1.0], labels=[0,1,2,3,4])
df

,Unnamed: 0,items,nodes,timesteps,arcs_ratio,islands,values,algorithm,scorefunction,recall,...,fscore,dist2opt,tp,tn,fp,fn,nodestates,arcs,items/nodestates,items/arcs
0,0,0.1,0.5,0.333333,0.499999,1,0.5,3OFF2,AIC,0.666667,...,3,0.333333,2,9,0,1,0.000244,0.1875,0.100000,0.1000
1,1,0.1,0.5,0.333333,0.499999,1,0.5,3OFF2,AIC,0.000000,...,0,1.414214,0,7,2,3,0.000244,0.1875,0.100000,0.1000
2,2,0.1,0.5,0.333333,0.499999,1,0.5,3OFF2,AIC,0.000000,...,0,1.414214,0,7,2,3,0.000244,0.1875,0.100000,0.1000
3,3,0.1,0.5,0.333333,0.499999,1,0.5,3OFF2,AIC,0.333333,...,1,0.833333,1,8,1,2,0.000244,0.1875,0.100000,0.1000
4,4,0.1,0.5,0.333333,0.499999,1,0.5,3OFF2,AIC,0.000000,...,0,1.414214,0,7,2,3,0.000244,0.1875,0.100000,0.1000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129785,129785,1.0,1.0,1.000000,1.000000,1,1.0,TABU,K2,0.527273,...,2,0.711878,29,464,33,26,1.000000,1.0000,0.000244,0.1875
129786,129786,1.0,1.0,1.000000,1.000000,1,1.0,TABU,K2,0.672727,...,2,0.580023,37,463,34,18,1.000000,1.0000,0.000244,0.1875
129787,129787,1.0,1.0,1.000000,1.000000,1,1.0,TABU,K2,0.890909,...,4,0.247555,49,483,14,6,1.000000,1.0000,0.000244,0.1875
129788,129788,1.0,1.0,1.000000,1.000000,1,1.0,TABU,K2,1.000000,...,4,0.214286,55,482,15,0,1.000000,1.0000,0.000244,0.1875


In [11]:
df3 = pd.get_dummies(df, columns=["algorithm","scorefunction"], dtype=int)
df3

,Unnamed: 0,items,nodes,timesteps,arcs_ratio,islands,values,recall,precision,fscore,...,algorithm_3OFF2,algorithm_HC,algorithm_MIIC,algorithm_MMHC,algorithm_TABU,scorefunction_AIC,scorefunction_BDEU,scorefunction_BIC,scorefunction_K2,scorefunction_L2L
0,0,0.1,0.5,0.333333,0.499999,1,0.5,0.666667,1.000000,3,...,1,0,0,0,0,1,0,0,0,0
1,1,0.1,0.5,0.333333,0.499999,1,0.5,0.000000,0.000000,0,...,1,0,0,0,0,1,0,0,0,0
2,2,0.1,0.5,0.333333,0.499999,1,0.5,0.000000,0.000000,0,...,1,0,0,0,0,1,0,0,0,0
3,3,0.1,0.5,0.333333,0.499999,1,0.5,0.333333,0.500000,1,...,1,0,0,0,0,1,0,0,0,0
4,4,0.1,0.5,0.333333,0.499999,1,0.5,0.000000,0.000000,0,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129785,129785,1.0,1.0,1.000000,1.000000,1,1.0,0.527273,0.467742,2,...,0,0,0,0,1,0,0,0,1,0
129786,129786,1.0,1.0,1.000000,1.000000,1,1.0,0.672727,0.521127,2,...,0,0,0,0,1,0,0,0,1,0
129787,129787,1.0,1.0,1.000000,1.000000,1,1.0,0.890909,0.777778,4,...,0,0,0,0,1,0,0,0,1,0
129788,129788,1.0,1.0,1.000000,1.000000,1,1.0,1.000000,0.785714,4,...,0,0,0,0,1,0,0,0,1,0


In [12]:
%%time
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    combinations = list(itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(1,len(s)+1)))
    
    merged = []
    for combination in combinations:
        merged.append(np.concatenate(combination, axis=None))
    return merged
    
item_c = ["items"]
nodes_c = ["nodes"]
arc_c = ["arcs_ratio"]
times_c = ["timesteps"]
values_c = ["values"]
algorithm_c = ["algorithm_3OFF2","algorithm_HC","algorithm_MIIC","algorithm_MMHC","algorithm_TABU"]
score_c = ["scorefunction_AIC","scorefunction_BDEU","scorefunction_BIC","scorefunction_K2","scorefunction_L2L"]
nodestates = ["nodestates"]
arcs = ["arcs"]
items_nodestates = ["items/nodestates"]
items_arcs = ["items/arcs"]


columns_c = [item_c, nodes_c, arc_c, times_c, values_c, algorithm_c, score_c, nodestates, arcs, items_nodestates, items_arcs]


column_sets = powerset(columns_c)

results3 = pd.DataFrame()
for columns in column_sets:
    # Splitting the data into training and testing data
    X_train, X_test, y_train, y_test = train_test_split(df3[columns], df3["fscore"], test_size = 0.2)

    regr = RandomForestRegressor()

    regr.fit(X_train, y_train)
        
    results3 = pd.concat([results3, pd.DataFrame({"items":["items" in columns], "nodes":["nodes" in columns], "arcs_ratio":["arcs_ratio" in columns], "timesteps":["timesteps" in columns], "values":["values" in columns], "algorithm":["algorithm_HC" in columns], "scorefunction":["scorefunction_AIC" in columns], "nodestates":["nodestates" in columns], "arcs":["arcs" in columns], "items/nodestates":["items/nodestates" in columns], "items/arcs":["items/arcs" in columns], "score":[regr.score(X_test, y_test)]})], ignore_index=True)

results3

CPU times: total: 3h 21min 29s
Wall time: 3h 23min 21s


,items,nodes,arcs_ratio,timesteps,values,algorithm,scorefunction,nodestates,arcs,items/nodestates,items/arcs,score
0,True,False,False,False,False,False,False,False,False,False,False,0.006982
1,False,True,False,False,False,False,False,False,False,False,False,0.085256
2,False,False,True,False,False,False,False,False,False,False,False,0.007130
3,False,False,False,True,False,False,False,False,False,False,False,0.066216
4,False,False,False,False,True,False,False,False,False,False,False,-0.000013
...,...,...,...,...,...,...,...,...,...,...,...,...
2042,True,True,True,False,True,True,True,True,True,True,True,0.300310
2043,True,True,False,True,True,True,True,True,True,True,True,0.343768
2044,True,False,True,True,True,True,True,True,True,True,True,0.357320
2045,False,True,True,True,True,True,True,True,True,True,True,0.348475


In [13]:
cm = sns.light_palette("red", as_cmap=True)

results3["items"] = results3["items"].astype(int)
results3["nodes"] = results3["nodes"].astype(int)
results3["arcs_ratio"] = results3["arcs_ratio"].astype(int)
results3["timesteps"] = results3["timesteps"].astype(int)
results3["values"] = results3["values"].astype(int)
results3["algorithm"] = results3["algorithm"].astype(int)
results3["scorefunction"] = results3["scorefunction"].astype(int)
results3["nodestates"] = results3["nodestates"].astype(int)
results3["arcs"] = results3["arcs"].astype(int)
results3["items/nodestates"] = results3["items/nodestates"].astype(int)
results3["items/arcs"] = results3["items/arcs"].astype(int)

results3.sort_values("score", ascending=False).to_csv("hotlinreg.csv")
results3.sort_values("score", ascending=False).head(10).style.background_gradient(cmap='RdYlGn', vmin=0, vmax=1)

,items,nodes,arcs_ratio,timesteps,values,algorithm,scorefunction,nodestates,arcs,items/nodestates,items/arcs,score
1635,1,0,1,1,0,1,1,0,1,0,1,0.390767
303,1,0,0,1,0,1,0,0,1,0,0,0.387435
719,1,0,0,1,0,1,1,0,0,0,1,0.382952
781,0,1,1,1,0,1,0,0,0,0,1,0.381802
599,1,1,0,1,0,1,0,0,0,0,1,0.381522
1501,1,1,1,1,0,1,1,0,1,0,0,0.381256
1430,0,0,1,1,0,1,0,0,1,1,1,0.380678
777,0,1,1,1,0,1,1,0,0,0,0,0.380242
1425,0,0,1,1,0,1,1,0,1,0,1,0.380080
1457,0,0,0,1,1,1,1,1,1,0,0,0.379868


# Bins - Discretized

In [14]:
df4 = df.copy()
df4['algorithmi'] = df4['algorithm'].map({'HC': 0, 'TABU':1, '3OFF2':2, 'MIIC':3, 'MMHC':4})
df4['scorefunctioni'] = df4['scorefunction'].map({'BDEU': 0, 'AIC':1, 'BIC':2, 'K2':3, 'L2L':4})
df4

,Unnamed: 0,items,nodes,timesteps,arcs_ratio,islands,values,algorithm,scorefunction,recall,...,tp,tn,fp,fn,nodestates,arcs,items/nodestates,items/arcs,algorithmi,scorefunctioni
0,0,0.1,0.5,0.333333,0.499999,1,0.5,3OFF2,AIC,0.666667,...,2,9,0,1,0.000244,0.1875,0.100000,0.1000,2,1
1,1,0.1,0.5,0.333333,0.499999,1,0.5,3OFF2,AIC,0.000000,...,0,7,2,3,0.000244,0.1875,0.100000,0.1000,2,1
2,2,0.1,0.5,0.333333,0.499999,1,0.5,3OFF2,AIC,0.000000,...,0,7,2,3,0.000244,0.1875,0.100000,0.1000,2,1
3,3,0.1,0.5,0.333333,0.499999,1,0.5,3OFF2,AIC,0.333333,...,1,8,1,2,0.000244,0.1875,0.100000,0.1000,2,1
4,4,0.1,0.5,0.333333,0.499999,1,0.5,3OFF2,AIC,0.000000,...,0,7,2,3,0.000244,0.1875,0.100000,0.1000,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129785,129785,1.0,1.0,1.000000,1.000000,1,1.0,TABU,K2,0.527273,...,29,464,33,26,1.000000,1.0000,0.000244,0.1875,1,3
129786,129786,1.0,1.0,1.000000,1.000000,1,1.0,TABU,K2,0.672727,...,37,463,34,18,1.000000,1.0000,0.000244,0.1875,1,3
129787,129787,1.0,1.0,1.000000,1.000000,1,1.0,TABU,K2,0.890909,...,49,483,14,6,1.000000,1.0000,0.000244,0.1875,1,3
129788,129788,1.0,1.0,1.000000,1.000000,1,1.0,TABU,K2,1.000000,...,55,482,15,0,1.000000,1.0000,0.000244,0.1875,1,3


In [15]:
%%time
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    combinations = list(itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(1,len(s)+1)))
    
    merged = []
    for combination in combinations:
        merged.append(np.concatenate(combination, axis=None))
    return merged
    
item_c = ["items"]
nodes_c = ["nodes"]
arc_c = ["arcs_ratio"]
times_c = ["timesteps"]
values_c = ["values"]
algorithm_c = ["algorithmi"]
score_c = ["scorefunctioni"]
nodestates = ["nodestates"]
arcs = ["arcs"]
items_nodestates = ["items/nodestates"]
items_arcs = ["items/arcs"]


columns_c = [item_c, nodes_c, arc_c, times_c, values_c, algorithm_c, score_c, nodestates, arcs, items_nodestates, items_arcs]


column_sets = powerset(columns_c)

results4 = pd.DataFrame()
for columns in column_sets:
    # Splitting the data into training and testing data
    X_train, X_test, y_train, y_test = train_test_split(df4[columns], df4["fscore"], test_size = 0.2)

    regr = RandomForestRegressor()

    regr.fit(X_train, y_train)
        
    results4 =  pd.concat([results4, pd.DataFrame({"items":["items" in columns], "nodes":["nodes" in columns], "arcs_ratio":["arcs_ratio" in columns], "timesteps":["timesteps" in columns], "values":["values" in columns], "algorithm":["algorithm_HC" in columns], "scorefunction":["scorefunction_AIC" in columns], "nodestates":["nodestates" in columns], "arcs":["arcs" in columns], "items/nodestates":["items/nodestates" in columns], "items/arcs":["items/arcs" in columns], "score":[regr.score(X_test, y_test)]})], ignore_index=True)

results4


KeyboardInterrupt



In [16]:
cm = sns.light_palette("red", as_cmap=True)

results4["items"] = results4["items"].astype(int)
results4["nodes"] = results4["nodes"].astype(int)
results4["arcs_ratio"] = results4["arcs_ratio"].astype(int)
results4["timesteps"] = results4["timesteps"].astype(int)
results4["values"] = results4["values"].astype(int)
results4["algorithm"] = results4["algorithm"].astype(int)
results4["scorefunction"] = results4["scorefunction"].astype(int)
results4["nodestates"] = results4["nodestates"].astype(int)
results4["arcs"] = results4["arcs"].astype(int)
results4["items/nodestates"] = results4["items/nodestates"].astype(int)
results4["items/arcs"] = results4["items/arcs"].astype(int)

results4.sort_values("score", ascending=False).to_csv("disclinreg.csv")
results4.sort_values("score", ascending=False).head(10).style.background_gradient(cmap='RdYlGn', vmin=0, vmax=1)

,items,nodes,arcs_ratio,timesteps,values,algorithm,scorefunction,nodestates,arcs,items/nodestates,items/arcs,score
389,0,1,0,1,0,0,0,0,0,0,1,0.375377
303,1,0,0,1,0,0,0,0,1,0,0,0.372059
305,1,0,0,1,0,0,0,0,0,0,1,0.368445
240,1,1,0,1,0,0,0,0,0,0,0,0.362664
352,0,1,1,1,0,0,0,0,0,0,0,0.359796
387,0,1,0,1,0,0,0,0,1,0,0,0.358937
183,0,0,0,1,0,0,0,0,1,0,0,0.355464
442,0,0,1,1,0,0,0,1,0,0,0,0.355441
385,0,1,0,1,0,0,0,0,0,0,0,0.351435
443,0,0,1,1,0,0,0,0,1,0,0,0.348930
